In [104]:
%load_ext autoreload
%autoreload 2

import collections
import cv2
import numpy as np
import os
import random
import reader 
import tensorflow as tf


class simple_learner(object):
    def __init__(self, num_classes, height=28, width=28):
        self.num_classes = num_classes
        self.height = height
        self.width = width
        
        self.tf_sess = tf.InteractiveSession()
    
    def network(self, learning_rate=0.00000005):
        # input and output
        i = tf.placeholder(tf.float32, shape=[None, self.height*self.width])
        self.i = i
        l = tf.placeholder(tf.float32, shape=[None, self.num_classes])
        self.l = l
        
        # add variables
        W = tf.Variable(tf.zeros([self.height*self.width, self.num_classes]))
        b = tf.Variable(tf.zeros([self.num_classes]))
        
        #initialize
        self.tf_sess.run(tf.initialize_all_variables())
        
        # create output and loss
        output = tf.nn.softmax(tf.matmul(i, W) + b)
        self.output = output
        
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(l * tf.log(output), reduction_indices=[1]))

        # training
        self.train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
        
        correct_prediction = tf.equal(tf.argmax(l,1), tf.argmax(output,1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    def test_accuracy(self, test_source):
        images, labels = test_source.all()
        return self.accuracy.eval(feed_dict={self.i: images, self.l: labels})
        
    def train(self, num_iterations, train_source, test_source):
        for i in xrange(num_iterations):
            if i % 500 == 0: 
                print i, self.test_accuracy(test_source)

            images, labels = train_source.next_batch(50)
            self.train_step.run(feed_dict={self.i: images, self.l: labels})
        
        print self.test_accuracy(test_source)
        
    def inference(self, images):
        return self.output.eval(feed_dict={self.i: images})


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [106]:
train = reader.reader('/home/lee/Desktop/rooms')
test = reader.reader('/home/lee/Desktop/rooms_test')


imgHeight = 28
imgWidth = 28
numClasses = train.num_classes()

learner = simple_learner(numClasses)
learner.network()
learner.train(2000, train, test)

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fc9b0957b50>> ignored


0 0.333333
500 0.555556
1000 0.555556
1500 0.577778
0.6


In [108]:
print "----TRAINING----"
train.print_classes()
print "----TESTING----"
test.print_classes()
print ""

from sklearn.metrics import confusion_matrix

images, labels = test.all()
output = tf.argmax(learner.inference(images), 1).eval()
labels = tf.argmax(labels, 1).eval()


print "----Confusion Matrix----"
confusion_matrix(output, labels)

----TRAINING----
dining_room 42
kitchen 44
living_room 57
----TESTING----
dining_room 15
kitchen 15
living_room 15

----Confusion Matrix----


array([[ 6,  2,  7],
       [ 4, 13,  0],
       [ 5,  0,  8]])